# chapter 4 - Web Mining Techniques
웹 마이닝 기법

<br></br>
## 1 웹 구조 마이닝
Web structure mining

### 01 웹 크롤러 - Spyder
### 02 인덱서- DB
### 03 페이지 랭킹 알고리즘 - Ranking (명성)
page I's in-link : 외부 페이지에서 I page 연결 수

page I's out-link : 페이지 I 가 연결하는 외부페이지 수
1. P(i) : i 페이지 방문확률
2. P(j) : j 페이지 방문확률
3. Aji: 노트 j에서 노드 i로 전이될 확률 
$$ P(i) = \sum_j A_{ji} P(j) $$

### 04 rank sing (랭크싱크) 
서로 링크한 경우에는 Loop에 빠져서, 랜덤한 점프 전이행렬을 추가
1. (1-d) 페이지를 임의로 방문할 확률
2. A는 확률로써 총 합은 1이다
$$ P = (\frac{(1-d)E}{N}+dA^T)P $$
3. 아래의 식 P벡터를 정규화 하면 간단해진다. $e^T P=N $
$$ P = (1-d)e + dA^TP \to P(i) = (1-d)+d\sum_{j=1}A_{ji}P(j) $$

<br></br>
## 2 웹 콘텐츠 마이닝
### 01 Parsing
파싱
### 02 자연어 처리
NLTK

Stemming : 단어를 어간으로 축소

In [1]:
text = """On 1 January 2007, Irish became a full EU official language, 
with a temporary derogation for a renewable period of five years (see Council 
Regulation (EC) No 920/2005 of 13 June 2005 (OJ L 156, 18.6.2005, p. 3)) stating 
that 'the institutions of the European Union shall not be bound by the obligation
to draft all acts in Irish and to publish them in that language in the Official 
Journal of the European Union """

In [2]:
from nltk.tokenize import WordPunctTokenizer
tknzr = WordPunctTokenizer()

words = tknzr.tokenize(text)
print('Words length : ', len(words), '\n\n', words[:20])

Words length :  91 

 ['On', '1', 'January', '2007', ',', 'Irish', 'became', 'a', 'full', 'EU', 'official', 'language', ',', 'with', 'a', 'temporary', 'derogation', 'for', 'a', 'renewable']


In [3]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

words_clean = [ w.lower()   for w in words   if w not in stopwords]
print('Clean words length : ', len(words_clean), '\n\n', words_clean[:15])

Clean words length :  64 

 ['on', '1', 'january', '2007', ',', 'irish', 'became', 'full', 'eu', 'official', 'language', ',', 'temporary', 'derogation', 'renewable']


In [4]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

words_clean_stem = [stemmer.stem(w)  for w in words_clean]
print('Clean words stem length : ', len(words_clean_stem), '\n\n', words_clean_stem[:15])
# token 의 갯수는 동일, 접사를 제거한 나머지를 출력

Clean words stem length :  64 

 ['on', '1', 'januari', '2007', ',', 'irish', 'becam', 'full', 'eu', 'offici', 'languag', ',', 'temporari', 'derog', 'renew']


<br></br>
## 3 정보 검색 모델
Information retrieval models

모델목록 : Boolean Model(불리언), Vector space model(벡터), Probabilistic model(확률)

Vector 모델 목록
1. TF-IDF(Term Frequency-Inverse Document Frequency) 단어빈도 - 역문서 빈도
2. LSA(Latent Semantic Analysis) 잠재의미분석
3. Doc2Vec (Word2Vec)

쿼리 단어간의 벡터로 표현하고, <strong>쿼리 벡터</strong>와 <strong>각 문서</strong>간의 <strong>$\cos \theta$ 유사도</strong> 를 측정, 비교한다 

### 01 TF-IDF
아주 <strong>많은 문서에 빈번</strong>하게 등장하는 단어는 <strong>중요도가 낮고</strong>, 
<strong>일부문서에 빈번</strong>한 단어의 <strong>중요도가 높다</strong>
$$ W_{ij} = tf_{ij}*idf_j $$
$ tf_{ij} = \frac {f_{ij}}{max f_{i1}...f_{iV}}$  문서 i에서 <strong>단어 j의 정규화 빈도</strong>

<strong>역문서 빈도</strong>의 $idf_j$는 단어 j를 포함하는 웹페이지 수, N은 전체 웹페이지 수이다 $ idf_j = log\frac{N}{df} $ 

### 02 잠재 의미 분석(LSA)
단어와 문서를 효과적으로 설명할 수 있는 정형적인 잠재공간이 존재하고, 비슷한 의미의 단어는 비슷한 위치에 나타난다

문서의 잠재공간 투영은 <strong>절단SVD</strong> (Singular Value Decomposition)를 이용한다
$$ X = U_t \sum_t V_t^T $$
$U_t(V*t)$ : t 차원의 잠재공간에 투영된 단어의 행렬

$\sum_t V_t^T $ : 잠재공간에 투영된 문서의 전치행렬

$\sum_{t(t*t)}$ : 특기앖으로 이루어진 대각행렬
$$ q_t = q^TU_t\sum_t^{-1}$$ 쿼리벡터도 잠재공간에 투영된다

각 문서는 <strong>$q_t^T$</strong>와 <strong>$\cos\theta$유사도</strong>로 비교될 수 있지만, 

실제는 $V_t^T$와 $q_t^T$간의 <strong>유사도</strong>로 계산한다

### 03 Word2Vec
두가지 아키텍처 알고리즘 중 하나를 선택하여, 

N개의 뉴런(Neuron)(가중치)으로 된 Hidden Layer를 훈련시킨다.

즉 N개의 뉴런(가중치) 'h'를 갖는 <strong>단일계층 학습</strong>으로 얕은학습(shallow learning)이다 

행렬W는 <strong>입력벡터를 hidden layer</strong>로 변환하며, <strong>출력계층에서는 Target</strong>을 평가한다

train에서 W와 W"의 수정은 <strong>확률적 내리막 경사법(Stochastic Gradient Descent)</strong>를 사용한다

https://shuuki4.wordpress.com/2016/01/27/word2vec-%EA%B4%80%EB%A0%A8-%EC%9D%B4%EB%A1%A0-%EC%A0%95%EB%A6%AC/

<h4><strong>연속단어 주머니 CBOW (Continuous Bag Of Words)</strong></h4>
<h5>ex) 아이스크림을 사 먹었는데, ___ 시려서 힘들었다. (빈칸 알맞은 단어 찾기)</h5>
주어진 단어 앞 뒤 C/2개 씩 <strong>총 C개 단어를 Input</strong>으로 사용, 

<strong>주어진 단어를 맞추기 위한 네트워크</strong>를 만든다
<h4>Input Layer 는</h4> 
모든 단어들이 공통적으로 사용하는 VxN (N:사용벡터의 길이(Projection Layer길이)) 

크기의 Projection Matrix $W$가 있고
<h4>Projection Layer에서 Output Layer로 갈 때는</h4> NxV 크기의 Weight Matrix W’ 가 있다.  
(주의해야할 점은, 두 행렬은 transpose이 아닌, 별개의 행렬이라는 점이다)
<h4>Process 정리</h4>
 1. 즉 단어들을 one-hot encoding으로 입력
2. 여러단어를 각각 projection 시킨 후, 벡터들의 평균을 구해서 Projection Layer에 보낸다. 
3. Weight Matrix를 곱해서 Output Layer로 보내고 softmax 계산을 한 후, 
4. 이 결과를 진짜 단어의 one-hot encoding과 비교하여 에러를 계산한다.

<h4>CBOW 모델에서 하나의 단어 처리 계산량</h4>
1. C개의 단어를 Projection 하는 데에 C x N
2. Projection Layer에서 Output Layer로 가는 데에 N x V, 전체 계산량은 CxN + NxV
3. 2.식은 V를 ln V로 줄이는 테크닉을 사용하면 전체 계산량이 CxN + N x lnV가 된다.

결국 <strong>Projection Layer의 크기 N</strong>과 <strong>log-사전크기의 lnV</strong>의 크기의 곱에 비례하게 된다.

C=10, N=500, V=1,000,000으로 잡아도 500 x (10+ln(1,000,000)) = 약 10000의 계산량밖에 들지 않는다. 

이는 앞서 확인한 NNLM이나 RNNLM에 비해 정말 엄청나게 줄어든 계산량이라는 것을 확인할 수 있다.

<img src="https://shuuki4.files.wordpress.com/2016/01/cbow.png?w=260&h=300" align="left">

<h4><strong>스킵그램(Skip-gram)</strong></h4>

주어진 단어 주위에 등장하는, 나머지 단어들의 등장 여부를 유추하는 것으로

<strong>‘가까이 위치한 단어일 수록 관련이 더 많다’</strong> 라는 생각을 적용하기 위해 

멀리 떨어져있는 단어일수록 낮은 확률로 택하는 방법을 사용한다. 

나머지 구조는 CBOW와 방향만 반대일 뿐 굉장히 유사하다

<h4>Skip-gram에서 하나의 단어를 처리하는 데에 드는 계산량</h4>

1. C개의 단어를 샘플링했다고 할 때, 현재 단어를 Projection 하는 N
2. Output을 계산하는 데에 N x V, 테크닉을 사용하면 N x ln V
3. 총 C개의 단어에 대해 진행해야 하므로 총 C배
4. 총 C(N + N x lnV) 만큼의 연산이 필요하다

CBOW 모델같이 N x lnV에 비례하는 계산량을 가진 모델이기는 하지만, 

샘플링 단어 갯수에 따라서 계산량이 비례하여 올라가게 되므로 CBOW 비해서는 느리다

실험결과, Skip-gram이 다소 좋은결과를 내는 추세를 보인다

<img src="https://shuuki4.files.wordpress.com/2016/01/skip-gram.png?w=276&h=300" align="left">

### 04 Doc2Vec
단어 $W_j$를 벡터 $V_w$로 표현하며, 단어가 나타나는 문서 $d_i$와는 독립적 벡터로 정의한다

Word2Vec알고리즘을 <strong>신경망</strong>과 <strong>역전파</strong>를 확장 이용한 것으로

문서를 단어의 벡터로 추가한 모델이다
<h4><strong>분산 메모리 모델 (DM) Distributed Memory model</strong>로써</h4>
문서 <strong>D</strong>벡터로 수집한 단어들이 학습모델과 공유되며, 

W와 W"는 모든 문서에 공통으로 연산하게 되어서 최적값을 찾는다 
<h4><strong>분산 단어 주머니(DBOW) distributed bag of words</strong> </h4>
https://radimrehurek.com/gensim/models/doc2vec.html

입력계층에서는 문서 벡터만 고려하고, 

출력계층은 문서 샘플링 Context의 단어 집합만 고려한다

DM아키텍처는 Gensim 라이브러리에 기본모듈로써 구현되어 있다
<h5>(아래는 3개 단어로 된 컨텍스트를 찾는 분산 메모리 모델의 예제이다)</h5>
<img src="https://research.google.com/archive/paragraph_vector_icml2014/distributed_memory_model.png" align="left" width="500">

<br></br>
## 4 영화 리뷰 쿼리분석 예제

### 01 데이터 불러오기
bs4 옵션설정 https://www.pythonanywhere.com/forums/topic/7002/  

open() utf-8 오류설정 https://stackoverflow.com/questions/20578270/unicodedecodeerror-in-python-while-reading-utf-8-sql-file-from-english-wikipedia  


In [5]:
import os
from time import time
import numpy as np
from bs4 import BeautifulSoup

movie_html_dir = './data/movie/'
movie_dict = {} # 데이터 파일 목록 저장

In [6]:
filenames = [f    for f in os.listdir(movie_html_dir)      if f[0]!='.'] ; t0 = time()
for file in filenames:
    id_html = file.split('.')[0]
    # UTF-8 오류발생시 외부 메세지 처리없이 결과를 출력한다
    f = open(movie_html_dir + file, encoding = "utf8", errors = 'replace')
    s = f.read()    
    parsed_html = BeautifulSoup(s,"lxml")
    try:     title = parsed_html.body.h1.text
    except:  title = 'none'
    movie_dict[id_html] = title
print('Web page :', len(filenames), '\nCrawling time :',round(time()-t0, 4))

Web page : 27886 
Crawling time : 42.491


### 02 긍/부정 Text 사전처리
nltk_movie_review

In [7]:
# 모듈호출 및 불용어 목록 불러오기
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer

tknzr = WordPunctTokenizer()
nltk.download('stopwords')
stoplist = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/markbaum/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# 어간처리 모듈 불러오기
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def ListDocs(dirname):
    docs, titles  = [], []
    filenames = [f for f in os.listdir(dirname) if str(f)[0]!='.']
    for filename in filenames:
        f = open(dirname+'/'+filename,'r')
        id_html = filename.split('.')[0].split('_')[1]
        titles.append(movie_dict[id_html])
        docs.append(f.read())
    return docs, titles

In [9]:
# 리뷰 텍스트 모두 메모리에 불러오기
dir = '/home/markbaum/nltk_data/corpora/movie_reviews/'
pos_textreviews, pos_titles = ListDocs(dir+'pos/')
neg_textreviews, neg_titles = ListDocs(dir+'neg/')
tot_textreviews = pos_textreviews + neg_textreviews
tot_titles = pos_titles + neg_titles

neg_textreviews[2][:500]

'david schwimmer ( from the television series " friends " ) stars as a sensitive ( and slightly neurotic ) single guy who gets more than he expected from the grieving mother ( barbara hershey ) of a classmate he can\'t remember . \nhello mrs . robinson ! \nthough quite cute as a romantic comedy , the pallbearer is paced like a funeral march . \nthe characters act , react , and interact at half-speed , making for one * excruciatingly * long sit . \n ( and what\'s with the dreary lighting ? ) \nco-writer/'

In [10]:
# 저장 데이터 목록 
# pos_titles
# neg_titles
# tot_titles

# pos_textreviews
# neg_textreviews
# tot_textreviews

### 03 TF-IDF 구현하기 sklearn 모듈 활용
TF-IDF model can be trained using sklearn

In [11]:
# Text 전처리 함수
def Preprocess_Tf_idf(texts, stoplist = [], stem = False):
    newtexts = []
    for text in texts:
        # 불용어 제거하기
        if stem:   tmp = [w for w in tknzr.tokenize(text) if w not in stoplist]
        # 유효한 단어 어간 처리 (접사 제거)
        else:      tmp = [stemmer.stem(w) for w in [w for w in tknzr.tokenize(text) if w not in stoplist]]
        newtexts.append(' '.join(tmp))
    return newtexts

# neg/pos 영화리뷰 데이터 전처리 하기
processed_reviews = Preprocess_Tf_idf(tot_textreviews, stoplist, True)

In [12]:
# tf-idf 모델 테스트 하기
t0 = time()
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df = 1)            # tf-idf 모듈 활성화
mod_tfidf = vectorizer.fit(processed_reviews)       # tf-idf 데이터 적용
vec_tfidf = mod_tfidf.transform(processed_reviews)  # tf-idf 벡터 변환
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

# http://hamait.tistory.com/803  # 반복 실행문의 부분실행 모듈
import itertools as it
print('tfidf\'s length :' , len(tfidf), '\n time :',round(time()-t0, 4))
{k: tfidf[k] for k in it.islice(tfidf.keys(), 10) }

tfidf's length : 39516 
 time : 1.3149


{'00': 6.4041777572475143,
 '000': 4.0688028414304771,
 '0009f': 7.908255154023788,
 '007': 6.2988172415896875,
 '00s': 7.908255154023788,
 '03': 7.908255154023788,
 '04': 7.908255154023788,
 '05': 7.2151079734638426,
 '05425': 7.908255154023788,
 '10': 3.3649603717537842}

In [13]:
# tf-idf 객체 저장하기
print ('학습한 html 리뷰 수:',len(processed_reviews),'\n--> TF-IDF 저장객체 :', len(mod_tfidf.get_feature_names()))
print ('\nTF-IDF :', mod_tfidf.get_feature_names()[::4000])
v = mod_tfidf.transform( processed_reviews )
print(v.shape)

# 학습한 객체를 저장한다
import  pickle  
with open('vectorizer.pk', 'wb') as file:
      pickle.dump(mod_tfidf, file)

# 저장한 객체를 불러온다
load_tfidf = pickle.load( open("vectorizer.pk", 'rb') )
load_tfidf.transform(Preprocess_Tf_idf([' '.join(['drama'])],stoplist,True))

학습한 html 리뷰 수: 2000 
--> TF-IDF 저장객체 : 39516

TF-IDF : ['00', 'blas', 'courier', 'establishment', 'hayakawa', 'lavishly', 'notoriety', 'rains', 'skim', 'transposes']
(2000, 39516)


<1x39516 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

### 04 gensim 의 LSA 모델 적용
단어 쿼리를 잠재공간으로 변환시 필요한 U,V,S 행렬을 얻는다

In [14]:
# Gensim 모델 만들기
import gensim
from gensim import models
class GenSimCorpus(object):
    def __init__(self, texts, stoplist=[],stem=False):
        self.texts = texts
        self.stoplist = stoplist
        self.stem = stem
        self.dictionary = gensim.corpora.Dictionary(self.iter_docs(texts, stoplist))
    def __len__(self):  return len(self.texts)
    def __iter__(self):
        for tokens in self.iter_docs(self.texts, self.stoplist):
            yield self.dictionary.doc2bow(tokens)
    def iter_docs(self,texts, stoplist):
        for text in texts:
            if self.stem: yield (stemmer.stem(w) for w in [x for x in tknzr.tokenize(text) if x not in stoplist])
            else:         yield (x for x in tknzr.tokenize(text) if x not in stoplist)

Using TensorFlow backend.


In [15]:
# corpus 생성하기
corpus = GenSimCorpus(tot_textreviews, stoplist, True)
dict_corpus = corpus.dictionary
print('corpus      :', corpus, '\ndict_corpus :', dict_corpus)

corpus      : <__main__.GenSimCorpus object at 0x7f79761a8eb8> 
dict_corpus : Dictionary(26132 unique tokens: ['film', 'unexpect', ',', 'scari', 'origin']...)


In [16]:
# U matrix 생성
n_topics = 10
lsi =  models.LsiModel(corpus, num_topics = n_topics, id2word = dict_corpus)
U = lsi.projection.u
U.shape

(26132, 10)

In [17]:
# Sigma 생성
Sigma = np.eye(n_topics) * lsi.projection.s
Sigma.shape

(10, 10)

In [18]:
# V 행렬을 계산한다
V = gensim.matutils.corpus2dense(lsi[corpus], len(lsi.projection.s)).T / lsi.projection.s
V.shape 

(2000, 10)

In [19]:
# 쿼리에 포함된 단어들의 색인'dict_words'을 생성한다
dict_words = {}

for i in range(len(dict_corpus)):
    dict_words[dict_corpus[i]] = i
    
print('dict_word\'s length :' , len(dict_words))
{k: dict_words[k] for k in it.islice(dict_words.keys(), 10) }

dict_word's length : 26132


{',': 2,
 '.': 9,
 'caught': 5,
 'film': 0,
 'guard': 6,
 'loop': 8,
 'origin': 4,
 'scari': 3,
 'threw': 7,
 'unexpect': 1}

### 05 gensim 의 Doc2Vec 를 생성 1
전처리 작업

In [20]:
# 전처리 함수
# token화, 어근처리(stemming), 불용어 제거
def Preprocess_Doc2Vec(text, stop = [], stem = False):
    words = tknzr.tokenize(text)  # text의 token 변화
    if stem:                      # 어근처리 (stemming)
        words_clean = [stemmer.stem(w)    for w in [ i.lower()    
                                                    for i in words   
                                                    if i not in stop]] # 불용어 제거
    else:
        words_clean = [i.lower()   for i in words   if i not in stop]  # 불용어 제거
    return words_clean

In [21]:
from collections import namedtuple
dir = '/home/markbaum/nltk_data/corpora/movie_reviews/'
Review = namedtuple('Review', 'words tags')

# 긍정적인 리뷰 데이터 수집하기
reviews_pos, cnt = [], 0  # 데이터 초기화 설정
do2vec_stem = False       # doc2vec의 어근처리 비활성화 (전처리 함수에서 미리 수행)

for filename in [f    for f in os.listdir(dir + 'pos/')    if str(f)[0]!='.']:
    f = open(dir + 'pos/' + filename, 'r')
    reviews_pos.append(Review(Preprocess_Doc2Vec(f.read(), stoplist, do2vec_stem), ['pos_' + str(cnt)]))
    cnt += 1    

In [22]:
# 부정적인 리뷰 데이터 수집하기
reviews_neg, cnt = [], 0  # 데이터 초기화 설정

for filename in [f    for f in os.listdir(dir + 'neg/')    if str(f)[0]!='.']:
    f = open(dir + 'neg/' + filename, 'r')
    reviews_neg.append(Review(Preprocess_Doc2Vec(f.read(), stoplist, do2vec_stem), ['neg_' + str(cnt)]))
    cnt += 1
tot_reviews = reviews_pos + reviews_neg

In [23]:
# 결과 
# reviews_pos  # 긍정 리뷰 1000개 모음
# reviews_neg  # 부정 리뷰 1000개 모음
# tot_reviews  # 모든 리뷰 모음
# word2vec 의 경우에는 do2vec_stem = False 즉 어근처리 않한경우에 더 결과가 좋은 경우가 많다

### 06 gensim 의 Doc2Vec 를 생성 2
Train 작업 by Gensim

In [24]:
# Doc2Vec 모델을 파라미터 설정
from gensim.models import Doc2Vec
import multiprocessing

numepochs= 20   # 훈련은 20시대 동안 진행된다
vec_size = 500  # 총 벡터의 수 : 은닉 레이어 갯수가 된다

In [25]:
# windos : 10개 단어의 관계망을 설정
# min_count : 최소 1번 언급된 단어는 모두 데이터를 생성
# negative  : 부정 샘플링
# hs : 계층적 소프트맥스 설정
cores = multiprocessing.cpu_count()
model_d2v = Doc2Vec(dm = 1, dm_concat = 0, size = vec_size, 
                    window = 10, negative = 0, hs = 0, min_count = 1, workers =cores)

# Doc2Vec 모델에 전체 리뷰데이터를 입력
model_d2v.build_vocab(tot_reviews)

In [27]:
# https://github.com/RaRe-Technologies/gensim/issues/1284
# .train : 개체의 갯수와, Epoch의 수를 명시적 입력
t0 = time
for epoch in range(numepochs):
    try:
        t0 = time()
        model_d2v.train(tot_reviews, total_examples = len(tot_reviews), epochs = numepochs)
        model_d2v.alpha *= 0.99  # 학습률
        model_d2v.min_alpha = model_d2v.alpha
        if epoch % 2 == 0: print ('epoch %d' % (epoch), round(time()-t0,3), 'sec')
    except (KeyboardInterrupt, SystemExit):
        break
print("word2vec train ends..", round(time()-t0)*10,4)

epoch 0 17.7211
epoch 2 20.2661
epoch 4 20.0804
epoch 6 21.5727
epoch 8 19.9699
epoch 10 20.041
epoch 12 19.9743
epoch 14 20.2278
epoch 16 20.0998
epoch 18 20.1269
word2vec train ends.. 20 4


### 07 TF-IDF 를 활용한 쿼리문 실행
mod_tfidf - Train 데이터 활용하기 1

In [28]:
# tf-idf를 활용하여 비슷한 웹페이지를 출력
query = ['drama','mystery','crime'] # 웹문서를 수집을 위한 query

# Similar tf-idf 결과값이 무척 작은경우 matrix 를 vector로 변환한다
from sklearn.metrics.pairwise import cosine_similarity

# 벡터를 정규 벡터로 변환
query_vec = mod_tfidf.transform(Preprocess_Tf_idf ([' '.join(query)], stoplist,  True))  
sims = cosine_similarity(query_vec, vec_tfidf)[0]  # tf-idf의 cos 유사도 측정 
indxs_sims = sims.argsort()[::-1]                  # 데이터 순서를 뒤집는다 (내림차순 정렬)
for d in list(indxs_sims)[:10]:
    print('sim:', sims[d],' title:', tot_titles[d])

sim: 0.216511203046  title: Usual Suspects, The (1995)
sim: 0.199655873219  title: Jane Austen's Mafia! (1998)
sim: 0.137757636005  title: Reservoir Dogs (1992)
sim: 0.135541633859  title: Score, The (2001)
sim: 0.12848238199  title: Mystery Men (1999)
sim: 0.127533806889  title: Se7en (1995)
sim: 0.12296714611  title: L.A. Confidential (1997)
sim: 0.113695734249  title: Lost Highway (1997)
sim: 0.107419256584  title: Very Bad Things (1998)
sim: 0.103080438187  title: True Crime (1999)


### 08 LSA의 $q_k$로 변환 쿼리문 활용하기
dict_words 목록 활용

In [29]:
# 웹문서를 수집을 위한 query
query = ['drama','mystery','crime']

# 쿼리문을 LSA의 qk로 변환
def Transform_WordsList_to_Query_Vec(wordslist, dict_words, stem = False):
    q = np.zeros(len( dict_words.keys()) )
    for w in wordslist:
        if stem:  q[dict_words[ stemmer.stem(w) ]] = 1.
        else:     q[dict_words[ w ]] = 1.
    return q

q  = Transform_WordsList_to_Query_Vec(query, dict_words, True)
print('q length :', len(q))
qk = np.dot(np.dot(q,U),Sigma); qk

q length : 26132


array([ 20.05895731,   0.52786705,  -1.35532556,   0.53508898,
         2.49284268,   1.80272746,  -0.34347263,  -0.6973278 ,
         1.64073059,  -0.82348602])

In [30]:
# 벡터를 정규벡터로 변환 뒤, 코사인 유사도를 계산
sims = np.zeros(len( tot_textreviews ))
for d in range(len(V)):
    sims[d]=np.dot(qk, V[d])

indxs_sims = np.argsort(sims)[::-1]  
for d in list(indxs_sims)[:10]:
    print ('sim : ', sims[d],' doc : ',tot_titles[d] )

sim :  1.59359220917  doc :  Rocky Horror Picture Show, The (1975)
sim :  1.51608878503  doc :  Alien³ (1992)
sim :  1.41083646704  doc :  Wild Things (1998)
sim :  1.40695314744  doc :  Star Wars: Episode I - The Phantom Menace (1999)
sim :  1.3501999207  doc :  Boogie Nights (1997)
sim :  1.29669576899  doc :  Deep Impact (1998)
sim :  1.2916246418  doc :  Black Cauldron, The (1985)
sim :  1.26846220112  doc :  Antz (1998)
sim :  1.26571395457  doc :  Starship Troopers (1997)
sim :  1.23622768048  doc :  Species II (1998)


### 09 doc2vec 모델에서 쿼리문 활용하기
model_d2v 모델링 활용

In [31]:
# 웹문서를 수집을 위한 query
query = ['love','sex','nudy']

# doc2vec query : force inference to get the same result
model_d2v.random = np.random.RandomState(1)
query_docvec = model_d2v.infer_vector(Preprocess_Tf_idf(' '.join(query), stoplist, do2vec_stem))
reviews_related = model_d2v.docvecs.most_similar([query_docvec], topn = 5)
reviews_related

# for review in reviews_related:
#     print('relevance:',review[1],'  title:',tot_titles[review[0]])

[('pos_418', 0.14885319769382477),
 ('neg_924', 0.14184680581092834),
 ('pos_48', 0.1287953108549118),
 ('neg_315', 0.11485865712165833),
 ('pos_197', 0.11204808950424194)]

결과

고급 알고리즘 보다 tf-idf 가 더 좋은 결과를 보여준다

Doc2Vec 등은 오히려 훈련데이터가 너무 작기 때문에 결과가 좋지 않다

http://www.cs.cornell.edu/people/pabo/movie-review-data/ 
는 10억개의 데이터로 훈련된 데이터를 릴리즈 하였다

<br></br>
## 5 잠재 디리클레 할당 - (사후 처리정보 1)
LDA (Latent Dirichlet allocation)

수집 데이터를 바탕으로, 다양한 정보를 추출하는 자연어 알고리즘

입력단어 <strong>(관측변수)</strong>는, <strong>잠재된 미관측변수(주제)</strong>에 의해 설명이 되고,

<strong>미관측 변수(주제)</strong>는 <strong>관측 데이터와 유사여부</strong>를 근거로 제공한다

즉 LDA는, 입력한 텍스트의 <strong>잠재주제 단어</strong>를 자동으로 찾고,

이를 바탕으로 문서의 <strong>공통주제 추출</strong>이 가능하다

이는 <strong>알고리즘의 '사후확률의 최대화'</strong>를 통해서 구현이 가능하다

### 01 데이터 수집 및 목록 만들기
train, DB 데이터 불러오기

In [34]:
# train 데이터 불러오기
# import os; import numpy as np
# from bs4 import BeautifulSoup
moviehtmldir = './data/movie/'
movie_dict = {}; t0=time()

for filename in [f for f in os.listdir(moviehtmldir) if f[0]!='.']:
    id_ = filename.split('.')[0]
    f = open(moviehtmldir+'/'+filename, encoding = "utf8", errors = 'replace')
    parsed_html = BeautifulSoup(f.read(), "lxml")
    try: title = parsed_html.body.h1.text
    except: title = 'none'
    movie_dict[id_] = title
print(round(time()-t0, 4) , 'sec')

37.797 sec


In [35]:
# 분석의 바탕이 되는 DB의 데이터 목록 불러오기
def ListDocs(dirname):
    docs, titles = [], []
    for filename in [f    for f in os.listdir(dirname)   if str(f)[0]!='.']:
        f = open(dirname + '/' + filename, 'r')
        id_file = filename.split('.')[0].split('_')[1]
        titles.append(movie_dict[id_file])
        docs.append(f.read())
    return docs, titles

dir = '/home/markbaum/nltk_data/corpora/movie_reviews/'
pos_textreviews,  pos_titles = ListDocs(dir + 'pos/')
neg_textreviews,  neg_titles = ListDocs(dir + 'neg/')
tot_textreviews = pos_textreviews + neg_textreviews
tot_titles = pos_titles + neg_titles

In [36]:
# 결과
# movie_dict : 색인용 목록 (위 for문과 아래의 dirlist 명령에 공통적으로 작업결과를 기록)
# http://hamait.tistory.com/803  # 반복 실행문의 부분실행 모듈
{k: movie_dict[k] for k in it.islice(movie_dict.keys(), 10) }

{'10378': 'Postman, The (1997)',
 '10807': 'Titanic (1997)',
 '11832': 'Lost in Space (1998)',
 '13684': 'Saving Private Ryan (1998)',
 '13764': 'Mark of Zorro, The (1920)',
 '1458': 'Revenge of the Nerds III: The Next Generation (1992) (TV)',
 '14726': 'Kundun (1997)',
 '29117': 'Others, The (2001)',
 '6389': 'Star Trek: First Contact (1996)',
 '6872': 'Azúcar amarga (1996)'}

### 02 데이터 전처리 및 LDA 모델 만들기
LDA 분석을 위한 전처리를 시행

In [37]:
# LDA 분석을 위한 전처리를 시행
from nltk.tokenize import RegexpTokenizer
tknzr = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps = True) # 전처리시 제거용 token 생성

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [38]:
# 불용어 제거함수
class GenSimCorpus(object):
    def __init__(self, texts, stoplist = [], bestwords = [], stem = False):
        self.texts = texts
        self.stoplist = stoplist
        self.stem = stem
        self.bestwords = bestwords
        self.dictionary = gensim.corpora.Dictionary(self.iter_docs(texts, stoplist))

    def __len__(self):
        return len(self.texts)
    def __iter__(self):
        for tokens in self.iter_docs(self.texts, self.stoplist):
            yield self.dictionary.doc2bow(tokens)
    def iter_docs(self,texts, stoplist):
        for text in texts:
            if self.stem: # 불용어를 제거한다
                yield (stemmer.stem(w) for w in [x for x in tknzr.tokenize(text) if x not in stoplist])
            else:
                if len(self.bestwords)>0:
                    yield (x for x in tknzr.tokenize(text) if x in self.bestwords)
                else:
                    yield (x for x in tknzr.tokenize(text) if x not in stoplist)            

In [39]:
# LDA 모델을 훈련시킨다
num_topics = 10; t0 = time()
corpus = GenSimCorpus(tot_textreviews, stoplist, [], False) # text 를 전처리 후 token을 걸러낸다
dict_lda = corpus.dictionary                                # 색인 용이한 {dict}로 객체를 변경한다

from gensim import models
lda = models.LdaModel(corpus, 
                      num_topics = num_topics, 
                      id2word = dict_lda,
                      passes = 10,
                      iterations = 50)
print(round(time()-t0, 4), 'sec')
lda.show_topics(num_topics = 3) # 완성된 LDA 모델을 test 한다

370.3987 sec


[(7,
  '0.000*" " + 0.000*"\n" + 0.000*"\'" + 0.000*"-" + 0.000*"one" + 0.000*"film" + 0.000*"movie" + 0.000*"like" + 0.000*"much" + 0.000*"good"'),
 (9,
  '0.206*" " + 0.015*"\n" + 0.012*"=" + 0.004*"\'" + 0.003*"-" + 0.001*"92s" + 0.001*"one" + 0.001*"film" + 0.001*"classic" + 0.000*"movie"'),
 (2,
  '0.726*" " + 0.037*"\n" + 0.017*"\'" + 0.009*"-" + 0.003*"film" + 0.002*"one" + 0.002*"movie" + 0.001*"like" + 0.001*"even" + 0.001*"time"')]

### 03  LDA 모델 다듬기
이상치 데이터 솎아내기

In [40]:
# 출현빈도 이상치를 갖는 데이터는 제거한다 (1000보다 많이 노출, 3보다 적게 노출된 데이터는 제거)
out_ids = [tokenid    for tokenid, docfreq  in  dict_lda.dfs.items() 
                      if docfreq > 1000 or docfreq < 3 ]
import copy
dict_lfq = copy.deepcopy(dict_lda)
dict_lfq.filter_tokens(out_ids)
dict_lfq.compactify()
print(dict_lfq)

Dictionary(18480 unique tokens: ['unexpected', 'scary', 'original', 'caught', 'guard']...)


In [41]:
corpus = [dict_lfq.doc2bow( tknzr.tokenize( text ))      for text in tot_textreviews]
print('Corpus:',len(corpus),'\n', corpus[0][:10])

Corpus: 2000 
 [(0, 1), (1, 2), (2, 5), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 6), (9, 1)]


### 03  LDA 모델 훈련 시키기
특정 주제에 해당되는 token 추출하기

In [42]:
# 10개 주제에 관련되어 모델을 훈련시킨다
# 결과는 주제별 나타날 확률이 가장높은 10개의 단어를 반환한다.
t0 = time()
lda_lfq = models.LdaModel(corpus, 
                          num_topics = num_topics,
                          id2word = dict_lfq,
                          passes = 10,
                          iterations = 50,
                          alpha = 0.01,
                          eta = 0.01)

for t in range(num_topics):
    print ('topic ',t,'  words: ',lda_lfq.print_topic(t,topn=10))
print(round(time()-t0, 4), 'sec')

topic  0   words:  0.003*"characters" + 0.003*"life" + 0.003*"see" + 0.003*"films" + 0.003*"love" + 0.003*"best" + 0.002*"little" + 0.002*"know" + 0.002*"really" + 0.002*"never"
topic  1   words:  0.007*"/" + 0.004*"really" + 0.004*"people" + 0.004*"see" + 0.003*"bad" + 0.003*"scene" + 0.003*"plot" + 0.003*"go" + 0.003*"another" + 0.003*"something"
topic  2   words:  0.010*"/" + 0.004*"see" + 0.004*"10" + 0.004*"life" + 0.003*"characters" + 0.003*"man" + 0.003*"scene" + 0.003*"little" + 0.003*"plot" + 0.002*"something"
topic  3   words:  0.003*"action" + 0.003*"plot" + 0.003*"scenes" + 0.003*"life" + 0.003*"/" + 0.002*"characters" + 0.002*"however" + 0.002*"great" + 0.002*"man" + 0.002*"best"
topic  4   words:  0.003*"films" + 0.003*"plot" + 0.003*"never" + 0.003*"many" + 0.003*"new" + 0.003*"life" + 0.003*"really" + 0.003*"could" + 0.003*"seems" + 0.003*"`"
topic  5   words:  0.004*"new" + 0.004*"characters" + 0.003*"star" + 0.003*"little" + 0.003*"life" + 0.003*"disney" + 0.003*"fami

In [43]:
lda_lfq.print_topic(t, topn=10)

'0.004*"alien" + 0.003*"films" + 0.003*"characters" + 0.003*"big" + 0.003*"really" + 0.003*"see" + 0.003*"movies" + 0.003*"star" + 0.002*"bad" + 0.002*"funny"'

In [44]:
# 실제 가장 높은 확률의 데이터 6위내 영화 쿼리를 추출한다
def GenerateDistrArrays(corpus):
    for i, dist in enumerate(corpus[:100]):  # corpus[:100] 모집단 데이터를 크게하면 된다 
        dist_array = np.zeros(num_topics)
        for d in dist:
            dist_array[d[0]] = d[1]
        if dist_array.argmax() == 6:
            print(tot_titles[i])

corpus_lda = lda_lfq[corpus]
GenerateDistrArrays(corpus_lda)

Hard Rain (1998)
Roommates (1995)
Matrix, The (1999)
Saving Private Ryan (1998)
Deuce Bigalow: Male Gigolo (1999)
Maximum Risk (1996)


In [45]:
# LDA 알고리즘의 군집화 결과를 보면, 
# 나름 유사한 영화끼리 군집화가 잘 되어있음을 알 수 있다

<br></br>
## 6 오피니언 마이닝 (감성분석) - (사후 처리정보 2)
글쓴 사람의 의견(긍정/ 중립/ 부정)을 추출하는 도구 : Opinion mining (sentiment analysis)

분류 알고리즘으로, 단어집의 갯수만큼 특징(feature)이 구성되므로, 

SVM/ Naives Bayes 등의 알고리즘을 사용할 수 있다

### 01 데이터 수집 및 목록 만들기
DB 데이터 불러오기

In [46]:
import nltk
from nltk.tokenize import WordPunctTokenizer
tknzr = WordPunctTokenizer()

from nltk.tokenize import RegexpTokenizer
tknzr = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps=True)

from nltk.corpus import stopwords
stoplist = stopwords.words('english')

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [47]:
from collections import namedtuple
def PreprocessReviews(text, stop = [], stem = False): #print profile
    words = tknzr.tokenize(text)
    if stem: words_clean = [stemmer.stem(w)   for w in [ i.lower() 
                                                         for i in words 
                                                         if i not in stop] ]
    else:    words_clean = [i.lower()    for i in words 
                                         if i not in stop ]
    return words_clean

Review = namedtuple('Review', 'words title tags')
dir = '/home/markbaum/nltk_data/corpora/movie_reviews/'
do2vecstem = True

In [49]:
# 긍정리뷰 데이터 수집하기
reviews_pos, cnt = [], 0
for filename in [f     for f in os.listdir(dir+'pos/')    if str(f)[0] != '.']:
    f = open(dir + 'pos/' + filename, 'r')
    id = filename.split('.')[0].split('_')[1]
    reviews_pos.append( Review( PreprocessReviews( f.read(),
                                                  stoplist, do2vecstem),
                                movie_dict[id],  ['pos_' + str(cnt)] ) )
    cnt += 1    

In [51]:
# 부정리뷰 데이터 수집하기
reviews_neg, cnt = [], 0
for filename in [f      for f in os.listdir(dir+'neg/')    if str(f)[0] != '.']:
    f = open(dir + 'neg/' + filename, 'r')
    id = filename.split('.')[0].split('_')[1]
    reviews_neg.append( Review( PreprocessReviews( f.read(),
                                                  stoplist, do2vecstem),
                               movie_dict[id], ['neg_'+str(cnt)] ) )
    cnt += 1

tot_reviews = reviews_pos + reviews_neg

### 02 수집 데이터로 Train/ Test 데이터 분류하기
Train/ Test 

In [52]:
# Train(80%)/ Test(20%) 데이터로 분리하기
def word_features(words):
    return dict([(word, True) for word in words])

neg_features = [(word_features(r.words), 'neg') for r in reviews_neg]
pos_features = [(word_features(r.words), 'pos') for r in reviews_pos]
portion_pos = int( len(pos_features) * 0.8 )
portion_neg = int( len(neg_features) * 0.8 )
print (portion_pos,'-->',portion_neg)
train_features = neg_features[:portion_neg] + pos_features[:portion_pos]
print (len(train_features))
test_features  = neg_features[portion_neg:] + pos_features[portion_pos:]
#shuffle(test_features)

800 --> 800
1600


In [53]:
# 나이브 베이즈 훈련모델 테스트
from nltk.classify import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_features)

print ('test on: ', len(test_features)); err = 0
for r in test_features:
    sent = classifier.classify(r[0])
    if sent != r[1]:    err += 1.
print ('error rate: ', err/float(len(test_features)))

test on:  400
error rate:  0.2875


In [59]:
# X^2 테스트를 통해서 높은 빈도로 발생한는 bi-gram '연어(collocation)' 목록을 추출한다 
# cf) 자연어에서의 '연어(collocation)' : 특정한 뜻을 나타낼 떄 함께 쓰이는 단어의 결합
# train 결과 오류가 28%로 상당히 높다
# 성능을 개선할 필요가 존재한다

### 03 $X^2$ 테스트를 활용
발생빈도가 높은 500개의 바이그램을 선별 후, Naive Bayes 분류기로 훈련시킨다

단어(i,j)로 된 bi-gram의 평균빈도 : $ O_{ij}$

bi-gram(i,j)의 기대빈도 : $ E_{ij}$
$$ X^2 = \frac{\sum_{i=0,j=0}^{1,1}(O_{ij}-E_{ij})^2}{E_{ij}} $$
ex) bi-gram 의 $X^2$을 계산한 경우 : $E = (\frac{O_{00}+O_{01}}{N} * \frac{O_{00}+O_{10}}{N})N $

case1) $X^2$이 커지면 : 평균빈도와 $ O_{ij}$ 기대빈도가 $ E_{ij}$ 벌어져서 

<strong>영가설이 기각</strong>되고, 이는 <strong>더 많은 정보를 포함한다</strong>는 결론에 도달한다

In [55]:
import itertools
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from random import shuffle

# Bi-gram 선별기
def bigrams_words_features(words, nbigrams = 200, measure = BigramAssocMeasures.chi_sq):
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(measure, nbigrams)
    return dict([(ngram, True) for ngram in itertools.chain(words, bigrams)])


In [60]:
# bi-gram 선별기로 token의 데이터 선별
neg_features = [(bigrams_words_features(r.words,500), 'neg') for r in reviews_neg]
pos_features = [(bigrams_words_features(r.words,500), 'pos') for r in reviews_pos]
portion_pos = int(len(pos_features)*0.8)
portion_neg = int(len(neg_features)*0.8)

train_features = neg_features[:portion_pos ] + pos_features[:portion_neg ]
print ('Pos_data:', portion_pos,'---> Nag_data:', portion_neg, 'Train :', len(train_features))
classifier = NaiveBayesClassifier.train(train_features)

Pos_data: 800 ---> Nag_data: 800 Train : 1600


In [58]:
## test bigram
test_features = neg_features[portion_neg:] + pos_features[portion_pos:]

from random import shuffle
shuffle(test_features)
err = 0
print( 'test on: ',len(test_features))
for r in test_features:
    sent = classifier.classify(r[0])
    #print r[1],'-pred: ',sent
    if sent != r[1]: err +=1.
print('error rate: ',err/float(len(test_features)))

test on:  400
error rate:  0.2025


In [ ]:
# 테스트 결과
# 빈도가 높은 500개를 선별해서 훈련한 결과
# 오차율이 20% 초반으로 낮아졌다, 즉 성능의 개선이 이루어졌다
# X^2 테스트는 가장 유익한 단어의 추출시 유용성이 증명되었다

In [ ]:
# X^2 에 대한 정리
# 단어의 중요도를 점수화 하기 위한 단어의 빈도가
# 긍정문서와 부정문서에서 어떻게 다른지를 측정한다
# ex) X^2값이 긍정리뷰서 높고, 부정리뷰서 낮은경우, 긍정리뷰의 정보에 해당


### 04 Corpus 를 정제후 테스트 실행
수집한 데이터 중 가장 유용한 10,000개 단어를 추출 후 학습한다

전체 corpus 데이터에서 총빈도, 긍정빈도, 부정빈도의 계산결과로 판단

In [65]:
import nltk.classify.util, nltk.metrics
tot_poswords = [val for l in [r.words for r in reviews_pos] for val in l]
tot_negwords = [val for l in [r.words for r in reviews_neg] for val in l]

from nltk.probability import FreqDist, ConditionalFreqDist
word_fd = FreqDist()
label_word_fd = ConditionalFreqDist(); t0 = time()
 
for word in tot_poswords:
    word_fd[word.lower()] +=1
    label_word_fd['pos'][word.lower()] +=1

for word in tot_negwords:
    word_fd[word.lower()] +=1
    label_word_fd['neg'][word.lower()] +=1
pos_words = len(tot_poswords)
neg_words = len(tot_negwords)
tot_words = pos_words + neg_words ; print(round(time()-t0,3), 'sec')

6.604 sec


In [82]:
# 주요한 긍정, 부정 데이터 중 주요정보를 갖는 corpus 10,000개를 선별한다
word_scores = {}; t0 = time()
 
for word, freq in word_fd.items():
    pos_score = BigramAssocMeasures.chi_sq(label_word_fd['pos'][word],
                (freq, pos_words), tot_words)
    neg_score = BigramAssocMeasures.chi_sq(label_word_fd['neg'][word],
                (freq, neg_words), tot_words)
    word_scores[word] = pos_score + neg_score
print ('total: ',len(word_scores))
best = sorted(word_scores.items(), key = lambda x : x[1], reverse=True)[:10000]
best_words = set([w for w, s in best]) ; print(round(time()-t0,3), 'sec')
{k for k in it.islice(best_words, 10) }

759.081 sec


{'bake',
 'biograph',
 'formal',
 'gosselaar',
 'gottfri',
 'hatchett',
 'mash',
 'purpos',
 'sensat',
 'underdevelop'}

In [84]:
# 정제한 결과를 활용하여 Naive Bayes 분류기를 훈련시킨다
# 훈련을 위해서 긍/부정 데이터 분류
def best_words_features(words):
    return dict([(word, True) for word in words if word in bestwords])

neg_features = [(best_words_features(r.words), 'neg') for r in reviews_neg]
pos_features = [(best_words_features(r.words), 'pos') for r in reviews_pos]
portion_pos = int(len(pos_features)*0.8)
portion_neg = int(len(neg_features)*0.8)
train_features = neg_features[:portion_pos] + pos_features[:portion_neg]
print (portion_pos,'-',portion_neg,'==>',len(train_features))

800 - 800 ==> 1600


In [87]:
classifier = NaiveBayesClassifier.train(train_features)

# 데이터를 '카이제곱 분포'로 test 한다
test_features = neg_features[portion_neg:] + pos_features[portion_pos:]
shuffle(test_features)
err = 0
print ('test on: ',len(test_features))
for r in test_features:
    sent = classifier.classify(r[0])
    # print(r[1],'-pred: ',sent)
    if sent != r[1]:  err +=1.
print ('error rate: ',err/float(len(test_features)))

test on:  400
error rate:  0.1125


In [ ]:
# 결과
# 테스트 결과 10,000개의 데이터 임에도, 오차율은 11%로 반이상 줄었다
# 이를 보다 신뢰하기 위한 결과를 위해서는, 교차 검증 방법을 활용해야 한다

### 05 정제 데이터를 활용한 Doc2Vec 분류
model_d2v.docvecs

In [88]:
# Doc2Vec 모델을 파라미터 설정
from gensim.models import Doc2Vec
import multiprocessing

num_epochs = 20
vec_size  = 500
shuffle(tot_reviews)
cores = multiprocessing.cpu_count()
model_d2v = Doc2Vec(dm = 1, dm_concat = 0, size = vec_size, window = 5, 
                    negative = 0, hs = 0, min_count = 1, workers = cores)

# 정의한 모듈에 데이터를 연셜하여 모델을 생성
model_d2v.build_vocab(tot_reviews)

In [93]:
# 훈련의 시작 
# https://github.com/RaRe-Technologies/gensim/issues/1284
t0 = time()
for epoch in range(numepochs):
    try:
        model_d2v.train(tot_reviews, total_examples = len(tot_reviews), epochs = num_epochs)
        model_d2v.alpha *= 0.99
        model_d2v.min_alpha = model_d2v.alpha
        if epoch % 2 == 0: print ('epoch :',epoch, round(time()-t0,3), 'sec')
    except (KeyboardInterrupt, SystemExit):
        break
print("word2vec train ends..", round(time()-t0)*(num_epochs/2), 4)
model_d2v.docvecs  # 훈련 데이터 저장객체

epoch : 0 20.554 sec
epoch : 2 62.735 sec
epoch : 4 104.905 sec
epoch : 6 146.174 sec
epoch : 8 187.566 sec
epoch : 10 229.758 sec
epoch : 12 271.316 sec
epoch : 14 312.834 sec
epoch : 16 354.581 sec
epoch : 18 396.102 sec
word2vec train ends.. 4170.0 4


In [94]:
# Train (80%), Test (20%) 데이터 분리하기
training_size = 2 * int( len( reviews_pos ) * 0.8)
train_d2v = np.zeros( (training_size, vec_size))
train_labels = np.zeros(training_size)

test_size = len(tot_reviews)-training_size
test_d2v = np.zeros((test_size, vec_size))
test_labels = np.zeros(test_size)

In [97]:
cnt_train, cnt_test = 0, 0
for r in reviews_pos:
    name_pos = r.tags[0]
    if int(name_pos.split('_')[1]) >= int(training_size/2.):
        test_d2v[cnt_test] = model_d2v.docvecs[name_pos]
        test_labels[cnt_test] = 1
        cnt_test +=1
    else:
        train_d2v[cnt_train] = model_d2v.docvecs[name_pos]
        train_labels[cnt_train] = 1
        cnt_train +=1

In [98]:
for r in reviews_neg:
    name_neg = r.tags[0]
    if int(name_neg.split('_')[1])>= int(training_size/2.):
        test_d2v[cnt_test] = model_d2v.docvecs[name_neg]
        test_labels[cnt_test] = 0
        cnt_test +=1
    else:
        train_d2v[cnt_train] = model_d2v.docvecs[name_neg]       
        train_labels[cnt_train] = 0
        cnt_train +=1

In [105]:
#train log regre
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
functions = [('LogisticReg', LogisticRegression() ), 
             ('SVC', SVC()),
             ('SVC(linear)', SVC(kernel = 'linear')),]

result = []
for name, clf in functions:
    t0 = time()
    clf.fit(train_d2v, train_labels)
    result.append([name, clf.score( test_d2v, test_labels), round(time()-t0, 3)])

import pandas as pd
df = pd.DataFrame(result, columns=['Name', 'Score', 'time'])
df.set_index('Name')

,Score,time
Name,,
LogisticReg,0.5275,0.037
SVC,0.5400,2.266
SVC(linear),0.5275,2.089


In [ ]:
# 결과
# 훈련결과 정확도 값들이 낮은데
# 이는 훈련데이터 집합의 크기가 작아서 나타난 결과이다